In [1]:
import pandas as pd
import sys
import json
import LLDA.model.labeled_lda as llda
import numpy as np
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine

# sys.path.append('../')

In [2]:
# # read the data
# data = pd.read_csv('FRIS_projects_pubs.csv')
# data.head()

# discipline_list = []
# for i in range(len(data)):
#     temp = data['disciplines'][i].replace("[", "").replace("]", "").replace("'", "").split(',')
#     temp = [x.strip() for x in temp]
#     discipline_list += temp

# discipline_list = list(set(discipline_list))

# print(sorted(discipline_list))
# print(len(discipline_list))

# # initialize data
# all_documents = []
# for i in range(len(data)):
#     temp = data['disciplines'][i].replace("[", "").replace("]", "").replace("'", "").split(',')
#     temp = [x.strip() for x in temp]
#     all_documents.append((data['title'][i], temp))

# split = int(len(all_documents)*0.8)
# labeled_documents, documents = all_documents[:split], all_documents[split:]
# labeled_documents[:5]

In [13]:
# # read json
# with open('FRIS_projects_pubs_abstracts.json') as f:
#     data = json.load(f)

# all_documents = []
# for entry in data:
#     entry = dict(entry)
#     uuid = list(entry.keys())[0]
#     if entry[uuid]['pub_abstracts']:
#         all_documents.append((entry[uuid]['project_abstract'], entry[uuid]['pub_abstracts']))

# split = int(len(all_documents)*0.8)
# labeled_documents, documents = all_documents[:split], all_documents[split:]
# labeled_documents[:5]

[('The relevance of the study of Latin & Greek (L&G) in secondary education is being questioned more and more. Proponents have claimed that it greatly improves linguistic abilities and even cognition beyond the linguistic domain, e.g. analytical thinking. Numerous American scholars have set up field tests, but they focused on the influence of Latin on English vocabulary acquisition by native speakers. Moreover, these studies all lack a theoretical framework for the supposed effects of L&G, so that the premises underlying hypotheses remain implicit. As a result, the debate on the educational value of L&G is still predominantly based on intuitive and ideologically-driven arguments rather than on empirical research. Therefore we propose a longitudinal study on the effects of L&G on Flemish pupils’ language competence (near transfer) and cognition (far transfer). Latin & Greek differ from the other languages standardly offered in Flemish secondary education because they are no longer spoke

In [4]:
# read json
with open('FRIS_projects_pubs_abstracts.json') as f:
    data = json.load(f)

df = pd.read_csv('FRIS_projects_pubs.csv')

all_documents = []
for entry in data:
    entry = dict(entry)
    uuid = list(entry.keys())[0]
    if entry[uuid]['pub_abstracts']:
        temp = df[df['pro_id'] == uuid]['disciplines'].values[0]
        temp = temp.replace("[", "").replace("]", "").replace("'", "").split(',')
        temp = [x.strip() for x in temp]
        all_documents.append((entry[uuid]['project_abstract'], temp))

split = int(len(all_documents)*0.8)
labeled_documents, documents = all_documents[:split], all_documents[split:]
labeled_documents[:5]

[('The relevance of the study of Latin & Greek (L&G) in secondary education is being questioned more and more. Proponents have claimed that it greatly improves linguistic abilities and even cognition beyond the linguistic domain, e.g. analytical thinking. Numerous American scholars have set up field tests, but they focused on the influence of Latin on English vocabulary acquisition by native speakers. Moreover, these studies all lack a theoretical framework for the supposed effects of L&G, so that the premises underlying hypotheses remain implicit. As a result, the debate on the educational value of L&G is still predominantly based on intuitive and ideologically-driven arguments rather than on empirical research. Therefore we propose a longitudinal study on the effects of L&G on Flemish pupils’ language competence (near transfer) and cognition (far transfer). Latin & Greek differ from the other languages standardly offered in Flemish secondary education because they are no longer spoke

In [5]:
len(labeled_documents)

460

In [2]:
# load from disk
llda_model = llda.LldaModel()
llda_model.load_model_from_dir("LLDA/data_models/pro_abstracts_disciplines", load_derivative_properties=False)
print("llda_model", llda_model)
# print("Top-5 terms of topic 'virus': ", llda_model.top_terms_of_topic("virus", 5, False))
print("Doc-Topic Matrix: \n", llda_model.theta)
print("Topic-Term Matrix: \n", llda_model.beta)

llda_model 
Labeled-LDA Model:
	K = 42
	M = 460
	T = 20377
	WN = 132085
	LN = 836
	alpha = 1.1904761904761905
	eta = 0.001
	perplexity = 852.0891495398791
	
Doc-Topic Matrix: 
 [[0.04599183 0.17687513 0.         ... 0.         0.         0.        ]
 [0.         0.         0.24232707 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.2961165  ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
Topic-Term Matrix: 
 [[9.00595023e-07 9.00595023e-07 9.00595023e-07 ... 9.00595023e-07
  9.00595023e-07 9.00595023e-07]
 [1.23399356e-06 1.23399356e-06 1.23399356e-06 ... 1.23399356e-06
  1.23399356e-06 1.23399356e-06]
 [4.20454789e-07 4.20454789e-07 4.20454789e-07 ... 4.20454789e-07
  4.20454789e-07 4.20454789e-07]
 ...
 [9.14597618e-07 9.14597618e-07 9.14597618e-07 ... 9.145

In [21]:
# llda_model.theta.shape
# 1 - pairwise_distances(llda_model.theta, metric="cosine")

(460, 1291)

In [13]:
# llda_model = llda.LldaModel(labeled_documents=labeled_documents)
# print(llda_model)


Labeled-LDA Model:
	K = 42
	M = 460
	T = 20377
	WN = 132085
	LN = 836
	alpha = 1.1904761904761905
	eta = 0.001
	perplexity = 968.6534305207664
	


In [15]:
# training
# llda_model.training(iteration=100, log=True)
# while True:
#     print("iteration %s sampling..." % (llda_model.iteration + 1))
#     llda_model.training(1)
#     print("after iteration: %s, perplexity: %s" % (llda_model.iteration, llda_model.perplexity()))
#     print("delta beta: %s" % llda_model.delta_beta)
#     if llda_model.is_convergent(method="beta", delta=0.01):
#         break

after iteration: 22, perplexity: 870.0354770072772
gibbs sample count:  132085
after iteration: 23, perplexity: 868.5819937848004
gibbs sample count:  132085
after iteration: 24, perplexity: 869.6295601174832
gibbs sample count:  132085
after iteration: 25, perplexity: 869.5437562486107
gibbs sample count:  132085
after iteration: 26, perplexity: 869.4989849936198
gibbs sample count:  132085
after iteration: 27, perplexity: 868.6138717233073
gibbs sample count:  132085
after iteration: 28, perplexity: 866.9250289282331
gibbs sample count:  132085
after iteration: 29, perplexity: 868.3025402312052
gibbs sample count:  132085
after iteration: 30, perplexity: 866.1864868845389
gibbs sample count:  132085
after iteration: 31, perplexity: 866.6590742357713
gibbs sample count:  132085
after iteration: 32, perplexity: 867.0894292052224
gibbs sample count:  132085
after iteration: 33, perplexity: 865.911075661152
gibbs sample count:  132085
after iteration: 34, perplexity: 866.3648719927676
gi

In [6]:
# # update
# print("before updating: ", llda_model)
# update_labeled_documents = [("new example test example test example test example test", ["example", "test"])]
# llda_model.update(labeled_documents=update_labeled_documents)
# print("after updating: ", llda_model)

# # train again
# # llda_model.training(iteration=10, log=True)
# while True:
#     print("iteration %s sampling..." % (llda_model.iteration + 1))
#     llda_model.training(1)
#     print("after iteration: %s, perplexity: %s" % (llda_model.iteration, llda_model.perplexity()))
#     print("delta beta: %s" % llda_model.delta_beta)
#     if llda_model.is_convergent(method="beta", delta=0.01):
#         break

In [13]:
# inference
# note: the result topics may be different for difference training, because gibbs sampling is a random algorithm
document = documents[0][0]
print(document)

topics = llda_model.inference(document=document, iteration=100, times=10)
# print sorted topics
sorted(topics, key=lambda x: x[0])

The IOF programme brings together expertise in (1) agricultural and food science, (2) sensor technologies and (3) statistics to help the agrifood industry through the fourth industrial revolution.  The main goal of the programme is to valorise the basic research in these disciplines together with relevant stakeholders.  Focus of the sensor-related work is nondestructive quality assessment using biophotonics.  The statistics work mainly focuses on advanced experimental design, multivariate statistics and statistical process control.


[('0101', 0.012848932676518885),
 ('0102', 0.013711001642036128),
 ('0103', 0.018883415435139578),
 ('0104', 0.011986863711001643),
 ('0105', 0.01026272577996716),
 ('0106', 0.012848932676518885),
 ('0107', 0.019745484400656814),
 ('0199', 0.011124794745484402),
 ('0201', 0.011124794745484402),
 ('0202', 0.011124794745484402),
 ('0203', 0.020607553366174057),
 ('0204', 0.01026272577996716),
 ('0205', 0.012848932676518885),
 ('0206', 0.012848932676518885),
 ('0207', 0.011124794745484402),
 ('0208', 0.011124794745484402),
 ('0299', 0.01026272577996716),
 ('0301', 0.01026272577996716),
 ('0302', 0.011124794745484402),
 ('0303', 0.011124794745484402),
 ('0304', 0.01026272577996716),
 ('0305', 0.013711001642036128),
 ('0306', 0.011124794745484402),
 ('0399', 0.01026272577996716),
 ('0401', 0.015435139573070609),
 ('0402', 0.01026272577996716),
 ('0499', 0.01026272577996716),
 ('0501', 0.011986863711001643),
 ('0502', 0.02750410509031199),
 ('0503', 0.01026272577996716),
 ('0504', 0.01371100

In [14]:
topic_matrix = np.zeros((len(documents), llda_model.K))
for i, (document, _) in enumerate(documents):
    topics = llda_model.inference(document=document, iteration=100, times=10)
    sorted_topics = sorted(topics, key=lambda x: x[0])
    for j, (topic, prob) in enumerate(sorted_topics):
        topic_matrix[i, j] = prob
print(topic_matrix)

[[0.01112479 0.01198686 0.013711   ... 0.02233169 0.01026273 0.43440066]
 [0.00594689 0.00825104 0.00548607 ... 0.00871187 0.00686855 0.52898837]
 [0.0070028  0.0170028  0.0070028  ... 0.00817927 0.00759104 0.55759104]
 ...
 [0.00954914 0.00715552 0.00396403 ... 0.00742148 0.01752786 0.69279382]
 [0.00248189 0.00385346 0.00372877 ... 0.00323002 0.00185845 0.69337965]
 [0.00668498 0.00572344 0.00860806 ... 0.01245421 0.00812729 0.5658196 ]]


In [17]:
# perplexity
# calculate perplexity on test data
perplexity = llda_model.perplexity(documents=map(lambda x: x[0], documents),
                                   iteration=30,
                                   times=10)
print("perplexity on test data: %s" % perplexity)
# calculate perplexity on training data
print("perplexity on training data: %s" % llda_model.perplexity())

perplexity on test data: 1115.2104641106398
perplexity on training data: 852.0902032423336


In [18]:
# save to disk
save_model_dir = "LLDA/data_models/pro_abstracts_disciplines"
# llda_model.save_model_to_dir(save_model_dir, save_derivative_properties=True)
llda_model.save_model_to_dir(save_model_dir)